## Train model identify if two images are the same turtle

### Import libraries

In [1]:
import os
import glob
import json
from os.path import join

import torchvision.datasets as datasets
import torchvision.transforms as transforms
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import random

from ShellRec.data_utils.prepare_photos import get_img_graph, split_graph
from ShellRec.training.shell_rec import TurtleDiff, TurtlePair, train

/home/yunyi/.conda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
random.seed(42)

### Preprocessing photo files

In [3]:
all_train = get_img_graph(path = "../dataset/BoxTurtle", drop_p=[0.95,0])
split_graph(all_train, save_path = "../dataset")
_ = get_img_graph(path = "../dataset/BoxTurtle_holdout", 
                  file_to_save = "../dataset/BoxTurtle_holdout.json")

### Prepare torch dataset

In [4]:
## image transformations
transform_train = timm.data.create_transform(224, is_training = True, 
                                   auto_augment = "rand-m9-mstd0.5")


transform_test = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.4850, 0.4560, 0.4060], 
                             std=[0.2290, 0.2240, 0.2250])
])

train_set = TurtlePair(data_file='../dataset/train.json',transform=transform_train)
val_set = TurtlePair(data_file='../dataset/val.json',transform=transform_test)

# Set up datasets and dataloaders
train_loader = DataLoader( TurtlePair(data_file='../dataset/train.json', 
                                             transform=transform_train), 
                                             batch_size = 128)
val_loader = DataLoader( TurtlePair(data_file='../dataset/val.json', 
                                           transform=transform_test), 
                                           batch_size = 128)

### training

Get the model set up, use a pretrained model as backbone, and add a new head to it.

In [5]:
torch.hub.set_dir('../pretrained/')

In [6]:
#device = "cpu" #
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   

#model = TurtleDiff('vit_base_patch16_384') # use a vit backbone
model = TurtleDiff('resnet50') # use a resnet backbone
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
num_epochs = 10

Training loop

In [7]:
train(model, optimizer, criterion, train_loader, val_loader, device, num_epochs,
      save_path = "../pretrained")

/home/yunyi/MIT/Turtle-Classifier/package/ShellRec/training/shell_rec.py:73: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(labels).to(device)
